In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

propublica_token = os.environ['propublica_token']

memberurl = 'https://voteview.com/static/data/out/members/HS117_members.csv'
cvoteurl = 'https://voteview.com/static/data/out/rollcalls/HS117_rollcalls.csv'
memvoteurl = 'https://voteview.com/static/data/out/votes/HS117_votes.csv'
cpartyurl = 'https://voteview.com/static/data/out/parties/HS117_parties.csv'

members_vv = pd.read_csv(memberurl)
cvote_vv = pd.read_csv(cvoteurl)
memvote_vv = pd.read_csv(memvoteurl)
cparty_vv = pd.read_csv(cpartyurl)

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

headers = {'X-API-Key': propublica_token,
          'User-Agent': useragent,
          'From': 'jkropko@virginia.edu'}

root = 'https://api.propublica.org'
congress = '117'
chamber = 'house'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)

r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
house_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

chamber = 'senate'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)
r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
senate_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

billsendpoint = '/congress/v1/{congress}/{chamber}/bills/{billtype}.json'.format(congress='117',
                                                                             chamber='both',
                                                                             billtype='introduced')
n = 20
i = 0
bills_117 = pd.DataFrame()
while n == 20:
    myparams = {'query': '', 'offset':i}
    r = requests.get(root + billsendpoint, headers = headers, params = myparams)
    myjson = json.loads(r.text)
    bills_pp = pd.json_normalize(myjson, record_path = ['results', 'bills'])
    bills_117 = pd.concat([bills_117, bills_pp], ignore_index=True)
    i = i + 20
    print(i)
    n = myjson['results'][0]['num_results']

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 600)

In [3]:
#members_vv.sort_values('nominate_dim1', ascending=True)

In [4]:
#members_vv.query("chamber == 'Senate' & born >= 1950 & born <= 1959")

bills_117.groupby('sponsor_name').size().sort_values(ascending=False)

bills_117['house_passage'][0] is not None

bills_117['pass_house'] = [x is not None for x in bills_117['house_passage']]
bills_117['pass_senate'] = [x is not None for x in bills_117['senate_passage']]
bills_117['pass_enact'] = [x is not None for x in bills_117['enacted']]

bills_117.groupby('sponsor_party').agg({'pass_house': 'mean',
                                       'pass_senate':'mean',
                                       'pass_enact':'mean'})

In [5]:
members_vv.head(3)

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,117,President,99912,99,0,USA,200,0.0,0.0,"TRUMP, Donald John",NaN,1946,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117,President,99913,99,0,USA,100,0.0,0.0,"BIDEN, Joseph Robinette, Jr.",B000444,1942,NaN,-0.320,0.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",R000575,1958,NaN,0.360,0.463,-74.23598,0.89924,699.0,24.0,NaN,0.422,0.539


In [6]:
house_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,D,None,RepAdams,CongresswomanAdams,None,412607,76386,5935,21545,N00035451,/m/02b45d,H4NC12100,https://adams.house.gov,https://adams.house.gov/rss.xml,None,True,None,-0.465,None,10,2022,925.0,3.0,0.0,2022-11-02 20:00:11 -0400,ocd-division/country:us/state:nc/cd:12,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,R,None,Robert_Aderholt,RobertAderholt,RobertAderholt,400004,45516,441,29701,N00003028,/m/024p03,H6AL04098,https://aderholt.house.gov,https://aderholt.house.gov/rss.xml,None,True,None,0.380,None,26,2022,925.0,14.0,0.0,2022-11-02 20:00:10 -0400,ocd-division/country:us/state:al/cd:4,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,D,House Democratic Caucus Vice Chair,reppeteaguilar,reppeteaguilar,None,412615,79994,70114,21506,N00033997,/m/0jwv0xf,H2CA31125,https://aguilar.house.gov,None,None,True,None,-0.296,None,8,2022,925.0,3.0,0.0,2022-11-02 20:00:11 -0400,ocd-division/country:us/state:ca/cd:31,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09


In [7]:
senate_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,B001230,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Tammy,None,Baldwin,None,1962-02-11,F,D,Senate Democratic Caucus Secretary,SenatorBaldwin,senatortammybaldwin,witammybaldwin,400013,57884,3470,29940,N00004367,/m/024v02,H8WI00018,https://www.baldwin.senate.gov,None,https://www.baldwin.senate.gov/feedback,True,None,-0.493,None,9,2024,866,1,0,2022-10-01 01:45:44 -0400,ocd-division/country:us/state:wi,709 Hart Senate Office Building,202-224-5653,None,WI,1,junior,S354,0.12,98.96,1.04
1,B001261,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Barrasso,None,1952-07-21,M,R,Senate Republican Conference Chair,SenJohnBarrasso,johnbarrasso,barrassowyo,412251,1024777,52662,40707,N00006236,/m/02rsm32,S6WY00068,https://www.barrasso.senate.gov,https://www.barrasso.senate.gov/public/?a=rss....,https://www.barrasso.senate.gov/public/index.c...,True,None,0.537,None,15,2024,866,58,0,2022-10-01 01:45:43 -0400,ocd-division/country:us/state:wy,307 Dirksen Senate Office Building,202-224-6441,None,WY,1,senior,S317,6.70,95.33,4.67
2,B001267,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Michael,None,Bennet,None,1964-11-28,M,D,None,SenBennetCo,senbennetco,SenatorBennet,412330,1031622,110942,40910,N00030608,/m/05b60qf,S0CO00211,https://www.bennet.senate.gov/public,None,https://www.bennet.senate.gov/public/index.cfm...,True,None,-0.229,None,13,2022,866,19,0,2022-10-12 09:45:23 -0400,ocd-division/country:us/state:co,261 Russell Senate Office Building,202-224-5852,202-228-5097,CO,3,senior,S330,2.19,99.06,0.94


In [8]:
members_pp = pd.concat([house_pp, senate_pp], ignore_index=True)
members_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,D,None,RepAdams,CongresswomanAdams,None,412607,76386,5935,21545,N00035451,/m/02b45d,H4NC12100,https://adams.house.gov,https://adams.house.gov/rss.xml,None,True,None,-0.465,None,10,2022,925.0,3.0,0.0,2022-11-02 20:00:11 -0400,ocd-division/country:us/state:nc/cd:12,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87,NaN,NaN,NaN
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,R,None,Robert_Aderholt,RobertAderholt,RobertAderholt,400004,45516,441,29701,N00003028,/m/024p03,H6AL04098,https://aderholt.house.gov,https://aderholt.house.gov/rss.xml,None,True,None,0.380,None,26,2022,925.0,14.0,0.0,2022-11-02 20:00:10 -0400,ocd-division/country:us/state:al/cd:4,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65,NaN,NaN,NaN
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,D,House Democratic Caucus Vice Chair,reppeteaguilar,reppeteaguilar,None,412615,79994,70114,21506,N00033997,/m/0jwv0xf,H2CA31125,https://aguilar.house.gov,None,None,True,None,-0.296,None,8,2022,925.0,3.0,0.0,2022-11-02 20:00:11 -0400,ocd-division/country:us/state:ca/cd:31,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09,NaN,NaN,NaN


In [9]:
members_pp.query("icpsr_id == '20301'")

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id
345,R000575,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Mike,D.,Rogers,None,1958-07-16,M,R,None,RepMikeRogersAL,CongressmanMikeDRogers,MikeRogersAL03,400341,1014740,5705,20301,N00024759,/m/024n_s,H2AL03032,https://mikerogers.house.gov,None,None,True,None,0.36,None,22,2022,925.0,37.0,0.0,2022-11-02 20:00:10 -0400,ocd-division/country:us/state:al/cd:3,2469 Rayburn House Office Building,202-225-3261,None,AL,3,False,0103,4.0,96.6,3.28,NaN,NaN,NaN


In [10]:
members_pp['icpsr'] = members_pp['icpsr_id'].astype('float')

In [11]:
members_total = pd.merge(members_vv, members_pp,
                        on = 'icpsr',
                        how = 'outer',
                        #validate = 'one_to_one',
                        indicator = 'matched')

In [12]:
members_total['matched'].value_counts()

both          385
right_only    171
left_only     170
Name: matched, dtype: int64

In [13]:
pd.set_option('display.max_columns', 80)
#members_total.query("matched == 'right_only'")[members_pp.columns]

In [14]:
members_pp.columns

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'date_of_birth', 'gender', 'party',
       'leadership_role', 'twitter_account', 'facebook_account',
       'youtube_account', 'govtrack_id', 'cspan_id', 'votesmart_id',
       'icpsr_id', 'crp_id', 'google_entity_id', 'fec_candidate_id', 'url',
       'rss_url', 'contact_form', 'in_office', 'cook_pvi', 'dw_nominate',
       'ideal_point', 'seniority', 'next_election', 'total_votes',
       'missed_votes', 'total_present', 'last_updated', 'ocd_id', 'office',
       'phone', 'fax', 'state', 'district', 'at_large', 'geoid',
       'missed_votes_pct', 'votes_with_party_pct', 'votes_against_party_pct',
       'senate_class', 'state_rank', 'lis_id', 'icpsr'],
      dtype='object')

In [15]:
members_vv.columns

Index(['congress', 'chamber', 'icpsr', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2'],
      dtype='object')

In [16]:
#members_pp.groupby(['last_name', 'date_of_birth']).size().sort_values(ascending=False)

In [17]:
members_vv[['bioname', 'born']].head(5)

,bioname,born
0,"TRUMP, Donald John",1946
1,"BIDEN, Joseph Robinette, Jr.",1942
2,"ROGERS, Mike Dennis",1958
3,"SEWELL, Terri",1965
4,"BROOKS, Mo",1954


In [18]:
members_pp[['last_name', 'date_of_birth']].head(5)

,last_name,date_of_birth
0,Adams,1946-05-27
1,Aderholt,1965-07-22
2,Aguilar,1979-06-19
3,Allen,1951-11-07
4,Allred,1983-04-15


1. Extract last name from bioname in members_vv
2. Convert last_name in members_pp to all caps
3. Extract the year from date_of_birth in members_pp
4. Check whether (last name, year) is unique in both members_vv and members_pp
5. If so, merge on (last name, year); If not, try with (last name, year, state)

In [19]:
members_vv['last_name'] = members_vv['bioname'].str.split(pat=',', expand=True)[0]

In [20]:
members_pp['last_name'] = members_pp['last_name'].str.upper()#.normalize('NFKD').encode('ascii', errors='ignore').str.decode('utf-8')
members_vv['last_name'] = members_vv['last_name'].str.upper()#.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [21]:
members_pp['last_name'] = members_pp['last_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
members_vv['last_name'] = members_vv['last_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [22]:
members_pp['born'] = members_pp['date_of_birth'].str.split(pat='-', expand=True)[0]

In [23]:
members_vv[['last_name', 'born', 'state_abbrev']].duplicated().value_counts()

False    555
dtype: int64

In [24]:
members_pp[['last_name', 'born', 'state']].duplicated().value_counts()

False    556
dtype: int64

In [25]:
members_vv['born'] = members_vv['born'].astype('str')
members_pp['born'] = members_pp['born'].astype('str')

In [26]:
members_vv.loc[members_vv['last_name'] == 'LEGER FERNANDEZ', 'last_name'] = 'FERNANDEZ'
members_pp.loc[members_pp['last_name'] == 'OMAR', 'born'] = '1982'
members_pp.loc[members_pp['last_name'] == 'PFLUGER', 'born'] = '1978'
members_vv.loc[members_vv['last_name'] == 'PFLUGER', 'born'] = '1978'
members_pp.loc[members_pp['last_name'] == 'FULCHER', 'born'] = '1962'

In [27]:
members_total = pd.merge(members_vv, members_pp,
                        left_on = ['last_name', 'born', 'state_abbrev'],
                        right_on = ['last_name', 'born', 'state'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')

In [28]:
members_total['matched'].value_counts()

both          553
right_only      3
left_only       2
Name: matched, dtype: int64

In [29]:
members_total.query("matched == 'right_only'")[['last_name', 'born', 'state']]

,last_name,born,state
555,PELTOLA,1973,AK
556,RYAN,1982,NY
557,SEMPOLINSKI,1983,NY


In [30]:
members_total.query("matched == 'left_only'")[['last_name', 'born', 'state_abbrev']]

,last_name,born,state_abbrev
0,TRUMP,1946,USA
1,BIDEN,1942,USA


In [31]:
members_total.iloc[556,22]==members_total.iloc[224,22]

False

In [32]:
members_total.iloc[556,11]

'1982'

In [33]:
members_total.iloc[224,11]

'1982'

In [34]:
members_total = pd.merge(members_vv, members_pp,
                        left_on = ['last_name', 'born', 'state_abbrev'],
                        right_on = ['last_name', 'born', 'state'],
                        how = 'right')

In [35]:
members_total.head(1).T

,0
congress,117.0
chamber,House
icpsr_x,21545.0
state_icpsr,47.0
district_code,12.0
state_abbrev,NC
party_code,100.0
occupancy,NaN
last_means,NaN
bioname,"ADAMS, Alma"


1. reduce columns in members data
2. get unique ID
2. get member vote and cvote tables too

In [38]:
cvote_vv.head(3)

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,117,House,1,2021-01-03,1,2,216,211,-0.057,0.038,0.707,0.274,-4.069,NaN,Pelosi,NaN,Election of the Speaker,NaN
1,117,House,2,2021-01-03,1,3,371,2,0.000,0.000,0.000,0.000,0.000,HRES1,Passed,Authorizing and directing the Speaker to admin...,On Agreeing to the Resolution,NaN
2,117,House,3,2021-01-04,1,4,214,204,0.019,-0.125,0.711,-0.235,-0.242,HRES8,Passed,Adopting the Rules of the House of Representat...,On Motion to Table the Motion to Postpone to a...,NaN


In [39]:
memvote_vv.head(3)

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,117,House,1,14066.0,6,100.0
1,117,House,1,14854.0,6,100.0
2,117,House,1,14863.0,6,99.6


In [43]:
members_total['icpsr_x'].isna().value_counts()

False    553
True       3
Name: icpsr_x, dtype: int64

In [44]:
members_total.columns

Index(['congress', 'chamber', 'icpsr_x', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2', 'last_name', 'id', 'title',
       'short_title', 'api_uri', 'first_name', 'middle_name', 'suffix',
       'date_of_birth', 'gender', 'party', 'leadership_role',
       'twitter_account', 'facebook_account', 'youtube_account', 'govtrack_id',
       'cspan_id', 'votesmart_id', 'icpsr_id', 'crp_id', 'google_entity_id',
       'fec_candidate_id', 'url', 'rss_url', 'contact_form', 'in_office',
       'cook_pvi', 'dw_nominate', 'ideal_point', 'seniority', 'next_election',
       'total_votes', 'missed_votes', 'total_present', 'last_updated',
       'ocd_id', 'office', 'phone', 'fax',